<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/23_AC_Replication/Sargassum_Replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Replicating carbon structure of this paper: https://www.mdpi.com/1420-3049/27/18/6040

In [21]:
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git

# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/


In [22]:
# Install required dependencies for GOPY
!apt-get update
!apt-get install -y libgl1-mesa-glx libxi6 libxrender1
!pip install numpy scipy

# Clone the GOPY repository and navigate into it
!git clone https://github.com/Iourarum/GOPY.git
%cd GOPY

# Generate a pristine graphene sheet
!python GOPY.py generate_PG 10 10 graphene.pdb

# Functionalize the graphene:
# Arguments: path_to_file, number_of_COOH, number_of_epoxy, number_of_OH, output_filename
!python GOPY.py generate_GO graphene.pdb 2 4 8 functionalized.pdb

# Return to the parent directory
%cd ..


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [29]:
!ls -l /content/lammps/build


total 109832
drwxr-xr-x  5 root root      4096 Jan 20 19:56 build-python
-rw-r--r--  1 root root     38544 Jan 20 19:43 CMakeCache.txt
drwxr-xr-x 57 root root      4096 Jan 20 19:56 CMakeFiles
-rw-r--r--  1 root root     15482 Jan 20 19:43 cmake_install.cmake
drwxr-xr-x  3 root root      4096 Jan 20 19:43 etc
drwxr-xr-x  3 root root      4096 Jan 20 19:43 includes
-rw-r--r--  1 root root       143 Jan 20 19:43 LAMMPSConfig.cmake
-rw-r--r--  1 root root      3258 Jan 20 19:43 LAMMPSConfigVersion.cmake
-rw-r--r--  1 root root       888 Jan 20 19:43 liblammps.pc
lrwxrwxrwx  1 root root        14 Jan 20 19:56 liblammps.so -> liblammps.so.0
-rwxr-xr-x  1 root root 111631264 Jan 20 19:56 liblammps.so.0
-rwxr-xr-x  1 root root     78040 Jan 20 19:56 lmp
-rw-r--r--  1 root root    662502 Jan 20 19:43 Makefile
drwxr-xr-x  2 root root      4096 Jan 20 19:43 styles


In [30]:
import os
os.environ["OMP_NUM_THREADS"] = "4"  # limit OpenMP threads if you want

import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.constants import Avogadro

!pip install MDAnalysis
import MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF

In [31]:
def append_HgCl2_to_graphene(input_file="/content/graphene.lmp", output_file="data_with_HgCl2.lmp", num_HgCl2=10):
    import numpy as np

    # Read the existing file
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Identify section indices
    sections = {}
    for keyword in ["Masses", "Atoms", "Bonds", "Angles"]:
        for i, line in enumerate(lines):
            if line.strip().startswith(keyword):
                sections[keyword] = i
                break

    # Extract header and counts
    header_lines = lines[:sections["Masses"]]
    current_atoms = current_bonds = current_angles = 0
    for line in header_lines:
        tokens = line.split()
        if len(tokens) == 2 and tokens[1] == "atoms":
            current_atoms = int(tokens[0])
        elif len(tokens) == 2 and tokens[1] == "bonds":
            current_bonds = int(tokens[0])
        elif len(tokens) == 2 and tokens[1] == "angles":
            current_angles = int(tokens[0])

    # Determine positions of sections
    masses_idx = sections["Masses"]
    atoms_idx = sections["Atoms"]
    bonds_idx = sections.get("Bonds", None)
    angles_idx = sections.get("Angles", None)

    # Determine end of Atoms section
    if bonds_idx:
        atoms_end = bonds_idx
    elif angles_idx:
        atoms_end = angles_idx
    else:
        atoms_end = len(lines)

    # Determine end of Bonds section
    if angles_idx:
        bonds_end = angles_idx
    else:
        bonds_end = len(lines)

    # Extract sections
    masses_lines = lines[masses_idx:atoms_idx]
    atoms_lines = lines[atoms_idx:atoms_end]
    if bonds_idx:
        if angles_idx:
            bonds_lines = lines[bonds_idx:angles_idx]
        else:
            bonds_lines = lines[bonds_idx:bonds_end]
    else:
        bonds_lines = []
    rest_lines = lines[angles_idx:] if angles_idx else []

    # Parse existing atoms
    existing_atoms = []
    for line in atoms_lines[2:]:
        if line.strip():
            parts = line.split()
            existing_atoms.append(parts)

    # Parse existing bonds
    existing_bonds = []
    if bonds_lines:
        for line in bonds_lines[2:]:
            if line.strip():
                parts = line.split()
                existing_bonds.append(parts)

    # Setup for new HgCl2 molecules
    atom_types = {'Hg': 4, 'Cl': 5}
    charges = {4: 2.0, 5: -1.0}
    bond_length_HgCl = 2.3

    next_atom_id = current_atoms + 1
    next_bond_id = current_bonds + 1
    next_mol_id = max(int(atom[1]) for atom in existing_atoms) + 1 if existing_atoms else 1

    xlo, xhi = 0.0, 100.0
    ylo, yhi = 0.0, 100.0
    zlo, zhi = 0.0, 35.0
    box_bounds = (xlo, xhi, ylo, yhi, zlo, zhi)

    def generate_random_position_box(box_bounds, existing_positions, min_distance=2.5, tolerance=0.1):
        max_attempts = 1000
        xlo, xhi, ylo, yhi, zlo, zhi = box_bounds
        for _ in range(max_attempts):
            x = np.random.uniform(xlo, xhi)
            y = np.random.uniform(ylo, yhi)
            z = np.random.uniform(zlo, zhi)
            pos = np.array([x, y, z])
            if existing_positions.size == 0:
                return pos
            distances = np.linalg.norm(existing_positions - pos, axis=1)
            if np.all(distances >= (min_distance - tolerance)):
                return pos
        return None

    existing_positions = np.array([[float(atom[4]), float(atom[5]), float(atom[6])] for atom in existing_atoms]) if existing_atoms else np.empty((0,3))
    new_atoms = []
    new_bonds = []

    for _ in range(num_HgCl2):
        Hg_pos = generate_random_position_box(box_bounds, existing_positions, min_distance=2.5)
        if Hg_pos is None:
            continue
        direction1 = np.random.randn(3); direction1 /= np.linalg.norm(direction1)
        direction2 = np.random.randn(3); direction2 /= np.linalg.norm(direction2)
        Cl1_pos = Hg_pos + bond_length_HgCl * direction1
        Cl2_pos = Hg_pos + bond_length_HgCl * direction2

        new_atoms.append(f"{next_atom_id} {next_mol_id} {atom_types['Hg']} {charges[4]:.4f} {Hg_pos[0]:.4f} {Hg_pos[1]:.4f} {Hg_pos[2]:.4f}\n")
        Hg_id = next_atom_id; next_atom_id += 1
        new_atoms.append(f"{next_atom_id} {next_mol_id} {atom_types['Cl']} {charges[5]:.4f} {Cl1_pos[0]:.4f} {Cl1_pos[1]:.4f} {Cl1_pos[2]:.4f}\n")
        Cl1_id = next_atom_id; next_atom_id += 1
        new_atoms.append(f"{next_atom_id} {next_mol_id} {atom_types['Cl']} {charges[5]:.4f} {Cl2_pos[0]:.4f} {Cl2_pos[1]:.4f} {Cl2_pos[2]:.4f}\n")
        Cl2_id = next_atom_id; next_atom_id += 1

        new_bonds.append(f"{next_bond_id} 1 {Hg_id} {Cl1_id}\n"); next_bond_id += 1
        new_bonds.append(f"{next_bond_id} 1 {Hg_id} {Cl2_id}\n"); next_bond_id += 1

        next_mol_id += 1
        existing_positions = np.vstack([existing_positions, Hg_pos, Cl1_pos, Cl2_pos])

    total_new_atoms = len(existing_atoms) + len(new_atoms)
    total_new_bonds = len(existing_bonds) + len(new_bonds)

    updated_header = []
    for line in header_lines:
        if "atoms" in line:
            updated_header.append(f"{total_new_atoms} atoms\n")
        elif "bonds" in line:
            updated_header.append(f"{total_new_bonds} bonds\n")
        else:
            updated_header.append(line)

    atoms_header = atoms_lines[:2]
    bonds_header = bonds_lines[:2] if bonds_lines else ["Bonds\n\n"]

    with open(output_file, 'w') as f:
        for line in updated_header:
            f.write(line)
        for line in masses_lines:
            f.write(line)
        for line in atoms_header:
            f.write(line)
        for atom in existing_atoms:
            f.write(" ".join(atom) + "\n")
        for line in new_atoms:
            f.write(line)
        f.write("\n")
        for line in bonds_header:
            f.write(line)
        for bond in existing_bonds:
            f.write(" ".join(bond) + "\n")
        for line in new_bonds:
            f.write(line)
        for line in rest_lines:
            f.write(line)

    print(f"Appended {num_HgCl2} HgCl2 molecules. New file saved as '{output_file}'.")


In [43]:
def setup_and_run_sim(num_Hg=50, functional_placement='edge'):
    # Use append_HgCl2_to_graphene to generate the data file with specified HgCl2 count
    data_file = f"data_{num_Hg}.lmp"
    append_HgCl2_to_graphene(input_file="/content/graphene.lmp", output_file=data_file, num_HgCl2=num_Hg)

    input_script = f"""
units           real
atom_style      full
boundary        p p p

read_data       data_20.lmp    # or data_40.lmp, data_60.lmp, etc.

# ------------------
# LONG-RANGE SETTINGS
# ------------------
pair_style      lj/cut/coul/long 12.0 12.0
kspace_style    pppm 1.0e-5

# -------------------
# PAIR COEFFICIENTS (9 types)
# -------------------
# Each line: pair_coeff type type epsilon sigma
# The lines below are placeholders; fill in correct values.

pair_coeff 1 1 0.07 3.40   # Type 1 (C4)
pair_coeff 2 2 0.07 3.40   # Type 2 (CX)
pair_coeff 3 3 0.07 3.40   # Type 3 (CY)
pair_coeff 4 4 0.07 3.40   # Type 4 (CZ)
pair_coeff 5 5 0.02 2.50   # Type 5 (HK, e.g. a hydrogen)
pair_coeff 6 6 0.16 3.05   # Type 6 (OE, an oxygen)
pair_coeff 7 7 0.16 3.05   # Type 7 (OJ, an oxygen)
pair_coeff 8 8 0.16 3.05   # Type 8 (OK, an oxygen)
pair_coeff 9 9 0.16 3.05   # Type 9 (OL, an oxygen)

pair_modify     mix arithmetic

# -------------------
# BOND/ANGLE/DIHEDRAL COEFFS
# -------------------
# Because the data file says "14 bond types, 31 angle types, 70 dihedral types,"
# we must define them all. These are *example dummy values*—replace them with
# correct force-field numbers for each type in your CGCMM or other FF.

bond_style      harmonic
# bond_coeff type K (kcal/mol/Å^2)   r0 (Å)
bond_coeff 1 400.0 1.40
bond_coeff 2 400.0 1.40
bond_coeff 3 400.0 1.40
bond_coeff 4 400.0 1.40
bond_coeff 5 400.0 1.40
bond_coeff 6 400.0 1.40
bond_coeff 7 400.0 1.40
bond_coeff 8 400.0 1.40
bond_coeff 9 400.0 1.40
bond_coeff 10 400.0 1.40
bond_coeff 11 400.0 1.40
bond_coeff 12 400.0 1.40
bond_coeff 13 400.0 1.40
bond_coeff 14 400.0 1.40

angle_style     harmonic
# angle_coeff type K (kcal/mol/rad^2)  theta0 (degrees)
angle_coeff 1  50.0 120.0
angle_coeff 2  50.0 120.0
angle_coeff 3  50.0 120.0
angle_coeff 4  50.0 120.0
angle_coeff 5  50.0 120.0
angle_coeff 6  50.0 120.0
angle_coeff 7  50.0 120.0
angle_coeff 8  50.0 120.0
angle_coeff 9  50.0 120.0
angle_coeff 10 50.0 120.0
angle_coeff 11 50.0 120.0
angle_coeff 12 50.0 120.0
angle_coeff 13 50.0 120.0
angle_coeff 14 50.0 120.0
angle_coeff 15 50.0 120.0
angle_coeff 16 50.0 120.0
angle_coeff 17 50.0 120.0
angle_coeff 18 50.0 120.0
angle_coeff 19 50.0 120.0
angle_coeff 20 50.0 120.0
angle_coeff 21 50.0 120.0
angle_coeff 22 50.0 120.0
angle_coeff 23 50.0 120.0
angle_coeff 24 50.0 120.0
angle_coeff 25 50.0 120.0
angle_coeff 26 50.0 120.0
angle_coeff 27 50.0 120.0
angle_coeff 28 50.0 120.0
angle_coeff 29 50.0 120.0
angle_coeff 30 50.0 120.0
angle_coeff 31 50.0 120.0

# Dihedral style can be harmonic, charmm, opls, class2, etc.
# For a CGCMM-style system, "class2" or "opls" is common. We'll pick "opls" as an example.
dihedral_style  opls


variable kd equal 2.0
variable zero equal 0.0
variable minus equal -2.0

# We'll just do the same line for all 70.
# e.g., dihedral_coeff n 2.0 0.0 0.0 0.0
# Adjust as needed for your real FF.

dihedral_coeff 1  {kd} 0.0 0.0 0.0
dihedral_coeff 2  {kd} 0.0 0.0 0.0
dihedral_coeff 3  {kd} 0.0 0.0 0.0
dihedral_coeff 4  {kd} 0.0 0.0 0.0
dihedral_coeff 5 {kd} 0.0 0.0 0.0
dihedral_coeff 6  {kd} 0.0 0.0 0.0
dihedral_coeff 7  {kd} 0.0 0.0 0.0
dihedral_coeff 8  {kd} 0.0 0.0 0.0
dihedral_coeff 9  {kd} 0.0 0.0 0.0
dihedral_coeff 10 {kd} 0.0 0.0 0.0
dihedral_coeff 11  {kd} 0.0 0.0 0.0
dihedral_coeff 12  {kd} 0.0 0.0 0.0
dihedral_coeff 13  {kd} 0.0 0.0 0.0
dihedral_coeff 14  {kd} 0.0 0.0 0.0
dihedral_coeff 15 {kd} 0.0 0.0 0.0
dihedral_coeff 16  {kd} 0.0 0.0 0.0
dihedral_coeff 17  {kd} 0.0 0.0 0.0
dihedral_coeff 18  {kd} 0.0 0.0 0.0
dihedral_coeff 19 {kd} 0.0 0.0 0.0
dihedral_coeff 20 {kd} 0.0 0.0 0.0
dihedral_coeff 21  {kd} 0.0 0.0 0.0
dihedral_coeff 22  {kd} 0.0 0.0 0.0
dihedral_coeff 23  {kd} 0.0 0.0 0.0
dihedral_coeff 24  {kd} 0.0 0.0 0.0
dihedral_coeff 25  {kd} 0.0 0.0 0.0
dihedral_coeff 26  {kd} 0.0 0.0 0.0
dihedral_coeff 27  {kd} 0.0 0.0 0.0
dihedral_coeff 28  {kd} 0.0 0.0 0.0
dihedral_coeff 29  {kd} 0.0 0.0 0.0
dihedral_coeff 30  {kd} 0.0 0.0 0.0
dihedral_coeff 31  {kd} 0.0 0.0 0.0
dihedral_coeff 32  {kd} 0.0 0.0 0.0
dihedral_coeff 33  {kd} 0.0 0.0 0.0
dihedral_coeff 34  {kd} 0.0 0.0 0.0
dihedral_coeff 35  {kd} 0.0 0.0 0.0
dihedral_coeff 36  {kd} 0.0 0.0 0.0
dihedral_coeff 37  {kd} 0.0 0.0 0.0
dihedral_coeff 38  {kd} 0.0 0.0 0.0
dihedral_coeff 39  {kd} 0.0 0.0 0.0
dihedral_coeff 40  {kd} 0.0 0.0 0.0
dihedral_coeff 41  {kd} 0.0 0.0 0.0
dihedral_coeff 42  {kd} 0.0 0.0 0.0
dihedral_coeff 43  {kd} 0.0 0.0 0.0
dihedral_coeff 44  {kd} 0.0 0.0 0.0
dihedral_coeff 45  {kd} 0.0 0.0 0.0
dihedral_coeff 46  {kd} 0.0 0.0 0.0
dihedral_coeff 47  {kd} 0.0 0.0 0.0
dihedral_coeff 48  {kd} 0.0 0.0 0.0
dihedral_coeff 49  {kd} 0.0 0.0 0.0
dihedral_coeff 50  {kd} 0.0 0.0 0.0
dihedral_coeff 51  {kd} 0.0 0.0 0.0
dihedral_coeff 52  {kd} 0.0 0.0 0.0
dihedral_coeff 53  {kd} 0.0 0.0 0.0
dihedral_coeff 54  {kd} 0.0 0.0 0.0
dihedral_coeff 55  {kd} 0.0 0.0 0.0
dihedral_coeff 56  {kd} 0.0 0.0 0.0
dihedral_coeff 57  {kd} 0.0 0.0 0.0
dihedral_coeff 58  {kd} 0.0 0.0 0.0
dihedral_coeff 59  {kd} 0.0 0.0 0.0
dihedral_coeff 60  {kd} 0.0 0.0 0.0
dihedral_coeff 61  {kd} 0.0 0.0 0.0
dihedral_coeff 62  {kd} 0.0 0.0 0.0
dihedral_coeff 63  {kd} 0.0 0.0 0.0
dihedral_coeff 64  {kd} 0.0 0.0 0.0
dihedral_coeff 65  {kd} 0.0 0.0 0.0
dihedral_coeff 66  {kd} 0.0 0.0 0.0
dihedral_coeff 67  {kd} 0.0 0.0 0.0
dihedral_coeff 68  {kd} 0.0 0.0 0.0
dihedral_coeff 69  {kd} 0.0 0.0 0.0
dihedral_coeff 70  {kd} 0.0 0.0 0.0

# If your data has 0 impropers, you can omit "improper_style" or set none:
improper_style none

# ------------------
# GROUPS & FIXES
# ------------------
# Example grouping of carbon. You can revise as needed:
group carbon type 1 2 3 4
fix   fix_carbon carbon setforce 0.0 0.0 0.0

# Optionally set velocities, thermostats, etc.
# velocity all create 300.0 12345 dist gaussian
# fix nvt all nvt temp 300.0 300.0 100.0

thermo          1000
thermo_style    custom step temp etotal press

dump            1 all atom 100 dump_20.lammpstrj
timestep        1.0
run             20000
"""
    in_file = f"in_{num_Hg}.lmp"
    with open(in_file, 'w') as f:
        f.write(input_script)

    # Use the symlink 'lmp' instead of the hardcoded path
    get_ipython().system(f"/content/lammps/build/lmp -in {in_file}")

In [ ]:
pair_coeff 1 1 0.07 3.40  # C4
pair_coeff 2 2 0.07 3.40  # CX
pair_coeff 3 3 0.07 3.40  # CY
pair_coeff 4 4 0.07 3.40  # CZ
pair_coeff 5 5 0.02 2.50  # HK
pair_coeff 6 6 0.16 3.05  # OE
pair_coeff 7 7 0.16 3.05  # OJ
pair_coeff 8 8 0.16 3.05  # OK
pair_coeff 9 9 0.16 3.05  # OL


In [44]:
def analyze_simulation(num_Hg=50, box_dims=(150.0, 150.0, 100.0), mass_adsorbent=1.0,
                       initial_counts=None):
    if initial_counts is None:
        initial_counts = {'Hg': num_Hg, 'Mg': 10, 'Zn': 10, 'Ca': 10}
    data_file = f"data_{num_Hg}.lmp"
    dump_file = f"dump_{num_Hg}.lammpstrj"
    if not os.path.exists(dump_file):
        print(f"Dump file {dump_file} not found. Skipping analysis.")
        return None, None, None, None, None

    u = mda.Universe(
        topology=data_file,
        trajectory=dump_file,
        topology_format='DATA',
        trajectory_format='LAMMPSDUMP'
    )

    ref_group = u.select_atoms('type 1')  # Carbon
    target_group = u.select_atoms('type 4')  # Hg

    r_min, r_max = 0.0, 12.0
    rdf_calc = InterRDF(ref_group, target_group, range=(r_min, r_max), nbins=120)
    rdf_calc.run()

    r = rdf_calc.bins
    rdf = rdf_calc.rdf
    peaks, _ = find_peaks(rdf)
    if len(peaks) > 0:
        first_peak = peaks[0]
        inverted = -rdf
        minima, _ = find_peaks(inverted, distance=10)
        cands = minima[minima > first_peak]
        if len(cands) > 0:
            r_cut = r[cands[0]]
        else:
            r_cut = r_max
    else:
        r_cut = r_max

    adsorbed = 0
    total_frames = 0
    for ts in u.trajectory:
        p_ref = ref_group.positions
        p_tg = target_group.positions
        tr_ref = cKDTree(p_ref)
        tr_tg = cKDTree(p_tg)
        sdm = tr_ref.sparse_distance_matrix(tr_tg, max_distance=r_cut, output_type='coo_matrix')
        col_ids = np.unique(sdm.col)
        adsorbed += len(col_ids)
        total_frames += 1

    avg_adsorbed = adsorbed / total_frames if total_frames > 0 else 0.0

    MW_Hg = 200.59
    q_e = (avg_adsorbed * MW_Hg) / mass_adsorbent * 1e3  # mg/g

    volume_A3 = box_dims[0] * box_dims[1] * box_dims[2]
    volume_L = volume_A3 * 1e-24
    N_initial = initial_counts['Hg']
    N_unads = N_initial - avg_adsorbed
    C_e = (N_unads * MW_Hg / Avogadro) / volume_L * 1e3  # mg/L

    return C_e, q_e, r, rdf, r_cut


In [34]:
import csv

In [45]:
Hg_counts = [20, 40, 60, 80]
Ce_values = []
qe_values = []
rdf_data = []

for Hg_num in Hg_counts:
    setup_and_run_sim(Hg_num, functional_placement='edge')
    Ce, q_e, r, rdf, r_cut = analyze_simulation(Hg_num, box_dims=(150, 150, 100),
                                                initial_counts={'Hg': Hg_num, 'Mg': 10, 'Zn': 10, 'Ca': 10})
    if Ce is not None:
        Ce_values.append(Ce)
        qe_values.append(q_e)
        rdf_data.append((Hg_num, r, rdf))

# Save simulation data to CSV
with open('sorption_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Hg_num', 'Ce (mg/L)', 'q_e (mg/g)'])
    for Hg_num, Ce, q_e in zip(Hg_counts, Ce_values, qe_values):
        writer.writerow([Hg_num, Ce, q_e])
if len(Ce_values) > 2:
    def langmuir(C, q_max, K_L):
        return (q_max * K_L * C) / (1 + K_L * C)

    params, _ = curve_fit(langmuir, Ce_values, qe_values, p0=[max(qe_values), 0.1])
    Ce_fit = np.linspace(min(Ce_values), max(Ce_values), 100)
    qe_fit = langmuir(Ce_fit, *params)

    plt.figure(figsize=(8, 6))
    plt.scatter(Ce_values, qe_values, c='b', label='Simulation Data')
    plt.plot(Ce_fit, qe_fit, 'r--', label='Langmuir Fit')
    plt.xlabel('C_e (mg/L)')
    plt.ylabel('q_e (mg/g)')
    plt.title('Hg Sorption Isotherm on Activated Carbon')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print("Langmuir fit parameters:")
    print(f"q_max= {params[0]:.2f} mg/g, K_L= {params[1]:.4f} L/mg")

plt.figure(figsize=(10, 6))
for Hg_num, r, rdf in rdf_data:
    plt.plot(r, rdf, label=f'Hg_num={Hg_num}')
plt.xlabel('Distance r (Å)')
plt.ylabel('g(r)')
plt.title('Radial Distribution Function (RDF)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Appended 20 HgCl2 molecules. New file saved as 'data_20.lmp'.


NameError: name 'kd' is not defined

In [36]:
with open("data_20.lmp") as f:
    for line in f:
        print(line, end="")


LAMMPS data file. CGCMM style. atom_style full generated by VMD/TopoTools v1.8 on Mon Jan 20 10:50:09 PST 2025
138 atoms
133 bonds
 168 angles
 301 dihedrals
 0 impropers
 9 atom types
 14 bond types
 31 angle types
 70 dihedral types
 0 improper types
 -9.032500 20.967500  xlo xhi
 -10.475000 19.525000  ylo yhi
 -15.000000 15.000000  zlo zhi

# Pair Coeffs
#
# 1  C4
# 2  CX
# 3  CY
# 4  CZ
# 5  HK
# 6  OE
# 7  OJ
# 8  OK
# 9  OL

# Bond Coeffs
#
# 1  C4-CY
# 2  C4-OJ
# 3  C4-OK
# 4  CX-CX
# 5  CX-CY
# 6  CX-CZ
# 7  CY-CY
# 8  CY-CZ
# 9  CY-OE
# 10  CY-OL
# 11  CZ-CZ
# 12  CZ-OE
# 13  HK-OK
# 14  HK-OL

# Angle Coeffs
#
# 1  C4-CY-CX
# 2  C4-CY-CY
# 3  C4-CY-CZ
# 4  C4-OK-HK
# 5  CX-CX-CX
# 6  CX-CX-CY
# 7  CX-CX-CZ
# 8  CX-CY-CX
# 9  CX-CY-CY
# 10  CX-CY-CZ
# 11  CX-CY-OE
# 12  CX-CY-OL
# 13  CX-CZ-CX
# 14  CX-CZ-CY
# 15  CX-CZ-CZ
# 16  CX-CZ-OE
# 17  CY-C4-OJ
# 18  CY-C4-OK
# 19  CY-CX-CY
# 20  CY-CX-CZ
# 21  CY-CY-CZ
# 22  CY-CY-OE
# 23  CY-CY-OL
# 24  CY-CZ-CY
# 25  CY-CZ-CZ
# 26  